In [48]:
import pandas as pd
from sklearn import svm
import numpy as np
from time import time
import matplotlib.pyplot as plt
import matplotlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import PolynomialFeatures

In [49]:
def get_data_noisysine():
    data = pd.read_csv("datasets/noisysine.csv").values
    points=[]
    for d in data[:, 0] :
        points.append(np.array([d]))
    return np.array(points), data[:, 1]

def get_data_hydrodynamics():
    data = pd.read_csv("datasets/hydrodynamics.csv").values
    points=[]
    for d in data[:, 0:-1] :
        points.append(np.array(d))
    return np.array(points), data[:, -1]


def split_data(points, labels):
    points_train, points_test, labels_train, labels_test = train_test_split(points, labels, train_size=0.8)  
    return points_train, labels_train, points_test,  labels_test

In [50]:
def plot(points, labels, w, degree):
    matplotlib.rcParams['figure.figsize'] = [5, 5]
    points_min = points.min() - 1
    points_max = points.max() + 1
    points_n = np.arange(points_min, points_max, 0.02).reshape(-1, 1)
    labels_n = PolynomialFeatures(degree=degree).fit_transform(points_n) @ w
    plt.scatter(points, labels)
    plt.plot(points_n, labels_n, color='blue')
    plt.xticks(())
    plt.yticks(())
    plt.show()

In [51]:
def inv(x, y, a):
    return np.linalg.inv(np.transpose(x) @ x + a *np.identity(x.shape[1])) @ np.transpose(x) @ y

def h(w, x, degree):
    return PolynomialFeatures(degree=degree).fit_transform(x) @ w
    
def polynomial_regression(points_train, labels_train, points_test,  labels_test,degree, a):
    points_train_f = PolynomialFeatures(degree=degree).fit_transform(points_train)
    w = inv(points_train_f, labels_train, a)
    labels_test_n = h(w, points_test, degree)
    score  = 1 - (np.sum((labels_test_n - labels_test) ** 2)) / (np.sum((labels_test - np.mean(labels_test)) ** 2))
    return w, score

def optimize(points_train, labels_train, points_test,  labels_test, degree):
    max_w = None
    max_score = None
    max_a = None
    for i in range(10):
        a = 10 ** (-i)
        w, score = polynomial_regression(points_train, labels_train, points_test,  labels_test, degree, a)
        if max_score is None or score > max_score:
            max_w = w
            max_score = score
            max_alpha = a
    return max_w, max_score, max_alpha

In [ ]:
points, labels = get_data_noisysine()
points_train, labels_train, points_test,  labels_test = split_data(points, labels)
for d in [1, 2,3 , 4, 5]:
    w, score, a = optimize(points_train, labels_train, points_test,  labels_test, d)
    print(str(d) + "(alpha=" + str(a) + "): " + str(score))
    plot(points, labels, w, d)

In [47]:
points, labels = get_data_hydrodynamics()
points_train, labels_train, points_test,  labels_test = split_data(points, labels)
for d in [1, 2]:
    w, score, a= optimize(points_train, labels_train, points_test,  labels_test, d)
    print(str(d) + "(alpha=" + str(a) + "): " + str(score))

1(alpha=0.1): 0.5999693646340136
2(alpha=0.001): 0.8942284961725057


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
